In [1]:
import numpy as np
from math import sqrt, exp
from matplotlib import pyplot as plt

In [2]:
def get_stages(yn, A, c, h, t, f):
    # f = function
    s = len(c) # number of stages
    m = len(yn) # number of differential equations in system
    k = np.zeros([s, m]) # initialize empty array for stages
    k[0] = f(t + c[0] * h, yn) # initial stage k1
    # Compute stages k2, k3, k4, ... 
    for i in range(1, s):
        sum_prevStages = sum([A[i][j] * k[j] for j in range(i)])
        k[i] = f(t + c[i] * h, yn + h * sum_prevStages) 
    return k

def runge_kutta_solver(yn, A, b, c, h, t, f):
    # yn = approx. sol. of prev. timepoint
    # A = spatial shift matrix
    # b = averaging weight vector of length s (s = #stages)
    # c = temporal shift vector of length s 
    s = len(c) # number of stages
    k = get_stages(yn, A, c, h, t, f)
    y_nplus1 = yn + h * sum([b[i] * k[i] for i in range(s)]) # compute approx. solution y of next time point
    return y_nplus1
    

def rk_time_stepper(y0, A, b, c, h, t0, tf, f):
    # Call the runge kutta solver for all time points
    time = np.arange(t0, tf+h, h);
    N = len(time) # total number of time steps
    y_all_timesteps = np.zeros([N, len(y0)]) # initialize empty array for solutions of all time steps
    y_all_timesteps[0] = y0 # initial condition
    for n in range(1, N):
        y_all_timesteps[n] = runge_kutta_solver(y_all_timesteps[n-1], A, b, c, h, time[n], f);
    return time, y_all_timesteps
    
    

In [3]:
# Adaptive step size
def adaptive_step_size(x, x_hat, TOL, Fs, p, h):
    l = np.linalg.norm(np.array([x[i] - x_hat[i] for i in range(len(x))])) # local error estimator
    # if the local error is higher than the tolerance, reject x (return false) and reduce the step size h
    if l > TOL: 
        h = h * (TOL/l)**(1/(1+p)) * Fs
        return False, h
    
    # if the local error is <= tolerance, accept x (return true) and propose new, larger h for next step
    if l <= TOL:
        h = h * (TOL/l)**(1/(1+p)) * Fs
        return True, h

In [4]:
# Runge Kutta Method with dynamic step-size adaptation: Embedded Runge-Kutta method
def embedded_runge_kutta_stepper(y0, A, b, c, b_hat, h, t0, tf, f, TOL, Fs, p):

    # initialize list for solutions of all time steps for y and reference y_hat and pass initial value
    y_all_timesteps = [y0]
    y_all_timesteps_hat = [y0]
    time = [t0] # list for all the time-points computed. Will be returned.
    t = t0 # intialize time iterator with starting time t0
    i = 0
    accepted = False
    while t <= tf:
        # Compute y and y_hat for different h until they are accepted (error below tolerance)
        y = runge_kutta_solver(y_all_timesteps[i], A, b, c, h, t, f);
        y_hat = runge_kutta_solver(y_all_timesteps_hat[i], A, b_hat, c, h, t, f);
        
        accepted, h = adaptive_step_size(y, y_hat, TOL, Fs, p, h)
        
        if accepted:
            print("Iteration %d: Accepted y_n+1. New h for next iteration: %f" %(i, h))
            # if the local error of the solution is <= tolerance, accept y and y_hat and add to list
            # else repeat computation with new h
            y_all_timesteps += [y]
            y_all_timesteps_hat += [y_hat]
            t += h
            time += [t]
            i += 1 # number of time steps (iterator for accessing list for prev. solution)
            
    return time, y_all_timesteps, y_all_timesteps_hat
    

In [5]:
A_DOPRI5=np.array([[0,0,0,0,0,0,0],[1/5,0,0,0,0,0,0],[3/40,9/40,0,0,0,0,0],[44/45,-56/15,32/9,0,0,0,0],[19372/6561,-25360/2187,64448/6561,-212/729,0,0,0],[9017/3168,-355/33,46732/5247,49/176,-5103/18656,0,0],[35/384.,0,500/1113,125/192,-2187/6784,11/84,0]])
b_DOPRI5=np.array([35/384.,0,500/1113,125/192,-2187/6784,11/84,0])
b_hat_DOPRI5=np.array([5179/57600,0,7571/16695,393/640,-92097/339200,187/2100,1/40])
c_DOPRI5=np.array([0,1/5.,3/10.,4/5,8/9,1,1])

In [6]:
# SYSTEM OF ODES
# x0 = np.array([-1, 1])
# x0 = np.array([1, 1])
# x0 = np.array([2, 0])

# N = 100
# h = 1/N
# t0 = 0
# tf = t0 + N*h

# 2 BODY PROBLEM
# Initial values
x0 = [0.5, 0, 0, sqrt(3)]

# Parameters
t0 = 0
tf = 4
h = 0.01


TOL = 0.1
Fs = 0.8
p = 4
# time, y, y_hat = embedded_runge_kutta_stepper(x0, A_DOPRI5, b_DOPRI5, c_DOPRI5, b_hat_DOPRI5, h, t0, tf, two_body_problem, TOL, Fs, p)

In [7]:
plt.plot(time, y)
# plt.plot(time, y_hat)
None

NameError: name 'time' is not defined